In [ ]:

import pdfminer
import pdfrw
import alive_progress
import statistics

import sys
import os
from os.path import exists
import shutil
from pdfminer.high_level import extract_text
 
directory = "/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/all_cla_pdfs/"
from pdfrw import PdfReader, PdfWriter, PageMerge

#directory to put the pdfs that have been split, ones not split, and irregular pdfs
vert_split_dir= '/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_pdfs2'
standard_pdfs = '/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/standard_pdfs'
outside_pdfs = '/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/outside_pdfs'
#os.mkdir(vert_split_dir)

#function to split pdf pages in half
def splitpage(src):
    """Split a page into two (left and right)"""
    # Yield a result for each half of the page
    for x_pos in (0, 0.5):
        yield PageMerge().add(src, viewrect=(x_pos, 0, 0.5, 1)).render()
 
#count of split pdfs and dictionary with totals after each folder is processed
split = 0
unsplit = 0
split_dict = {}
unsplit_dict = {}
#get list of folders from relevant directory that has pdfs needing to be split 
jc_numbers= os.listdir(directory)
print(jc_numbers)
for dir in jc_numbers:
    print(dir)
    for document in os.listdir(f"{directory}/{dir}"):
        #split the pdf per line, count the characters
        if exists(f"{directory}/{dir}/{document}"):
            text = extract_text(f"{directory}/{dir}/{document}")
            text = text.splitlines()
            x=[]
            #rule out lines with less than 40 characters
            for i in text:
                if len(i)>40:
                    x.append(len(i))
            #take the average and median
            try: 
                avg_len_line = round(statistics.mean(x), 2)
            except:
                 avg_len_line = 100
            try:
                med_len_line = statistics.median(x)
            except:
                med_len_line = 100
                print ('Error on file:',document,'\n')
            #count the maximum number of characters on one line
            try:
                max_len_line = max(x)
            #if all lines are under 40 characters, its an abnormal pdf
            except:
                print('all max < 40')
                if os.path.exists(f"{outside_pdfs}/{dir}"):
                    shutil.copy(f"{directory}/{dir}/{document}", f"{outside_pdfs}/{dir}/{document}")
                    print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
                    print('pdf outside' + str(document))    
                else : 
                    os.mkdir(f"{outside_pdfs}/{dir}")
                    shutil.copy(f"{directory}/{dir}/{document}", f"{outside_pdfs}/{dir}/{document}")
                    print('pdf outside' + str(document))
            #make a PDF writer object
            writer = PdfWriter()
            #if the max length is above 67 and average and median lengths are less than 55, its a split pdf
            #run the splitting function and put this pdf in a split pdf directory from its PC name
            if max_len_line < 67 or (max_len_line > 67 and avg_len_line < 55 and med_len_line < 55) :
                page_number = 0
                for page in PdfReader(f"{directory}/{dir}/{document}").pages:
                    writer.addpages(splitpage(page))
                if os.path.exists(f"{vert_split_dir}/{dir}"):
                    if os.path.exists(f"{vert_split_dir}/{dir}/{document}"):
                        pass
                    else:
                        writer.write(f"{vert_split_dir}/{dir}/{document}")
                else:
                    os.mkdir(f"{vert_split_dir}/{dir}")
                    writer.write(f"{vert_split_dir}/{dir}/{document}")
                    print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
                split += 1    
            elif max_len_line > 67 and avg_len_line > 55 and med_len_line > 55:
                if os.path.exists(f"{standard_pdfs}/{dir}"):
                    if os.path.exists(f"{vert_split_dir}/{dir}/{document}"):
                        pass
                    else:
                        shutil.copy(f"{directory}/{dir}/{document}", f"{standard_pdfs}/{dir}/{document}")
                else:
                    os.mkdir(f"{standard_pdfs}/{dir}")
                    shutil.copy(f"{directory}/{dir}/{document}", f"{standard_pdfs}/{dir}/{document}")
                    print(str(avg_len_line) + ',' + str(med_len_line) + ',' + str(max_len_line))
                unsplit += 1
        else:
            print('error at directory: ' + str(dir))
    #keep track of splits and regulars in dicionary        
    split_dict[dir] = split 
    unsplit_dict[dir] = unsplit          
    print(f"Split pdfs: {split}")
    print(split_dict)
    print(f"regular pdfs: {unsplit}")
    print(unsplit_dict)

with open('/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_regular_dictionaries/split_dict.txt','a') as data: 
      data.write(str(split_dict))
with open('/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_regular_dictionaries/regular_dict.txt','a') as data: 
      data.write(str(unsplit_dict))

In [3]:
split_dict = {'210': 0, '110': 0, '121': 19, '149.02': 69, '202.01': 99, '202': 140, '200': 173, '112': 224, '102.08': 240, '106.01': 261, '315.02': 292, '105': 327, '324': 360, '148': 379, '109': 380, '115': 398, '315.01': 418, '315.03': 427, '322.01': 453, '309': 462, '319.01': 468, '224': 509, '100': 539, '129': 553, '219': 581, '102.07': 581, '326': 598, '140.01': 607, '125.02': 641, '140': 792, '301.05': 792, '140.03': 814, '339.03': 820, '209': 894, '217': 907, '207': 988, '116': 1066, '211': 1088, '145': 1162, '102.03': 1165, '336': 1186, '104': 1186, '203': 1190, '314': 1207, '332': 1214, '316': 1217, '124': 1288, '225': 1288, '144': 1367, '312': 1417, '328.02': 1417, '113': 1434, '225.02': 1439, '215': 1440, '301.01': 1504, '143': 1515, '313': 1533, '133': 1563, '320': 1580, '310': 1599, '325': 1610, '102.05': 1622, '146': 1625, '220': 1691, '331': 1699, '337': 1702, '303.01': 1707, '317': 1763, '339.02': 1772, '303.03': 1786, '102.06': 1810, '329.01': 1815, '323': 1851, '120.02': 1851, '120': 1863, '340': 1886, '214': 1897, '327.02': 1899, '140.05': 1899, '328.01': 1899, '225.01': 1912, '102.01': 1913, '125.01': 1948, '339.01': 1957, '223': 1962, '319.02': 1989, '149.01': 2041, '102.02': 2058, '113.04': 2083, '322': 2157, '226': 2205, '125': 2206, '152.01': 2223, '140.02': 2227, '139': 2240, '117': 2260, '107': 2274, '301': 2291, '149.04': 2347, '102.04': 2365, '300': 2365, '127': 2386, '321': 2412, '152': 2412, '329.02': 2429, '302': 2447, '333': 2473, '201': 2502, '304': 2516, '341': 2531, '222': 2554, '111': 2653, '318.02': 2654, '130': 2729, '142': 2730, '118': 2927, '126': 2962, '152.02': 2962, '140.04': 2979, '216': 3000, '318.01': 3002, '114': 3031, '327.03': 3034, '311': 3080, '142.02': 3106, '142.03': 3131, '102.09': 3135, '132': 3158, '142.01': 3200, '301.02': 3201, '319': 3210, '329.03': 3215, '120.01': 3228, '136': 3250, '106.02': 3284, '149.03': 3318, '330': 3400, '227': 3429, '221': 3443, '125.03': 3478, '327.01': 3486, '329': 3507, '306': 3523, '327': 3524, '120.03': 3540, '119': 3593, '106.03': 3594, '128': 3633, '328.03': 3673, '307': 3688, '142.04': 3730}
unsplit_dict = {'210': 23, '110': 72, '121': 99, '149.02': 99, '202.01': 99, '202': 99, '200': 99, '112': 100, '102.08': 102, '106.01': 102, '315.02': 102, '105': 106, '324': 106, '148': 106, '109': 162, '115': 180, '315.01': 180, '315.03': 180, '322.01': 184, '309': 185, '319.01': 225, '224': 227, '100': 227, '129': 233, '219': 233, '102.07': 260, '326': 266, '140.01': 267, '125.02': 271, '140': 283, '301.05': 312, '140.03': 313, '339.03': 313, '209': 325, '217': 325, '207': 330, '116': 336, '211': 336, '145': 343, '102.03': 355, '336': 355, '104': 378, '203': 382, '314': 386, '332': 441, '316': 446, '124': 446, '225': 447, '144': 463, '312': 463, '328.02': 489, '113': 496, '225.02': 499, '215': 548, '301.01': 552, '143': 621, '313': 621, '133': 622, '320': 625, '310': 625, '325': 625, '102.05': 632, '146': 638, '220': 638, '331': 694, '337': 715, '303.01': 715, '317': 715, '339.02': 717, '303.03': 719, '102.06': 743, '329.01': 767, '323': 774, '120.02': 793, '120': 825, '340': 826, '214': 853, '327.02': 875, '140.05': 877, '328.01': 885, '225.01': 907, '102.01': 926, '125.01': 927, '339.01': 930, '223': 937, '319.02': 963, '149.01': 963, '102.02': 966, '113.04': 967, '322': 975, '226': 975, '125': 975, '152.01': 993, '140.02': 993, '139': 1022, '117': 1022, '107': 1031, '301': 1035, '149.04': 1035, '102.04': 1036, '300': 1094, '127': 1103, '321': 1103, '152': 1103, '329.02': 1112, '302': 1146, '333': 1147, '201': 1147, '304': 1158, '341': 1163, '222': 1173, '111': 1189, '318.02': 1264, '130': 1269, '142': 1269, '118': 1275, '126': 1308, '152.02': 1325, '140.04': 1325, '216': 1328, '318.01': 1375, '114': 1378, '327.03': 1422, '311': 1422, '142.02': 1422, '142.03': 1422, '102.09': 1426, '132': 1429, '142.01': 1430, '301.02': 1448, '319': 1472, '329.03': 1472, '120.01': 1495, '136': 1510, '106.02': 1510, '149.03': 1511, '330': 1587, '227': 1587, '221': 1592, '125.03': 1594, '327.01': 1668, '329': 1673, '306': 1676, '327': 1682, '120.03': 1695, '119': 1695, '106.03': 1726, '128': 1726, '328.03': 1729, '307': 1730, '142.04': 1730}
with open('/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_regular_dictionaries/split_dict.txt','a') as data: 
      data.write(str(split_dict))
with open('/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_regular_dictionaries/regular_dict.txt','a') as data: 
      data.write(str(unsplit_dict))

In [ ]:
#if running functions after eachother this path is not needed, can be split_dir
#but otherwise make this path 
path = "/home/sammich/Documents/BeCode/becode_projects/KPMG---CLA-processing/data/split_pdfs2"

#join pdf half pages into full pdfs, in this case by language
pc_list = os.listdir(path)
for pc_folder in os.listdir(path):
    for document in os.listdir(f"{path}/{pc_folder}"):
        writer_NL = PdfWriter()
        writer_FR = PdfWriter()
        page_number = 0
        for page in PdfReader(f"{path}/{pc_folder}/{document}").pages:
            page_number += 1
            if page_number%2 == 1:
                writer_NL.addpage(page)
            else:
                writer_FR.addpage(page)
        try:
            os.mkdir('./data/split_nl/' + str(document)[0:3])
            os.mkdir('./data/split_fr/' + str(document)[0:3])
            writer_NL.write(f"./data/split_nl/" + str(document)[0:3] + f"/NL_{document}")
            writer_FR.write(f"./data/split_fr/" + str(document)[0:3]+ f"/FR_{document}")
        except:
            writer_NL.write(f"./data/split_nl/" + str(document)[0:3] + f"/NL_{document}")
            writer_FR.write(f"./data/split_fr/" + str(document)[0:3]+ f"/FR_{document}")
        else:
            print(('document' + str(document) + 'not made'))
            pass         

In [ ]:
# 326
# 50.55,52.0,59
# 63.9,61.5,107